## Импорт библиотек

In [0]:
!git clone https://github.com/sberbank-ai/ru-gpts
!mkdir models/

In [0]:
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

## Обучение модели

In [0]:
#!g1.1
%%time
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python3 ru-gpts/pretrain_transformers.py \
    --output_dir=models/gpt3_ten_epoch \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=../data/dataset/train.txt \
    --do_eval \
    --evaluate_during_training \
    --eval_data_file=../data/dataset/valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 4 \
    --logging_steps 1000 \
    --save_steps 3000 \
    --block_size 2048 \
    --overwrite_output_dir

## Результаты обученной модели


In [0]:
#!g2.1
def generate_text(text: str, path_to_model: str='models/gpt3_small/checkpoint-3000', max_length: int=50):
    
    tokenizer = GPT2Tokenizer.from_pretrained(path_to_model)
    model = GPT2LMHeadModel.from_pretrained(path_to_model)
    model.cuda()
    inpt = tokenizer.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(),
                     max_length=max_length, 
                     repetition_penalty=5.0, 
                     do_sample=True, 
                     top_k=5, 
                     top_p=0.95, 
                     temperature=0.8,
                     stop_token='\n')
    out_decoded = tokenizer.decode(out[0])
    answer = out_decoded[len(text):]
    return answer.strip()

## Примеры


Пример №1

In [0]:
#!g1.1
generate_text('Q: выдержит ли подставка напечанная на 3д принтре я станцию?')

Пример №2

In [0]:
#!g1.1
generate_text("Q: А как колонку подключить к Интернету?", max_length=30)

In [0]:
#!g1.1
generate_text("Q: Почему у меня не работает колонка?")

In [0]:
#!g1.1
generate_text("Q: Где купить Яндекс.Станцию?")

In [0]:
#!g1.1
